# 체크포인터를 사용한 상태관리

In [1]:
# 랭그래프의 체크포인터 시스템은 상태 영속성과 오류 복구를 위한 핵심 기능입니다. 
# 상태 영속성이란 그래프 실행 중 각 노드의 상태를 저장한다는 의미입니다. 
# 여러 대화나 세션의 상태를 독립적으로 관리할 수 있어서 동시에 여러 워크플로를 처리할 수 있습니다. 
# 또한 시스템 장애 시에도 마지막 체크포인트부터 재시작을 할 수 있습니다. 
# 이전에는 대화의 이력을 저장하려면 별도로 코드 작업을 해야 했는데, 
# 랭그래프에서는 이런 작업을 설정 한 줄로 끝낼 수 있도록 지원하고 있습니다.

In [2]:
# 사용 방법
#  1. 체크포인터 설정
#  2. 그래프에 체크포인터 연결
#  3. 스레드ID로 상태 관리

#  나머지는 랭그래프에서 알아서 동작시켜줌.

In [3]:
# 예시
"""
from langgraph.checkpoint.sqlite import SqliteSaver
# pip install langgraph-checkpoint-sqlite 필요

from langgraph.graph import StateGraph

# 1. 체크포인터 설정
checkpointer = SqliteSaver.from_conn_string(":memory:")

# 2. 그래프에 체크포인터 연결
app = StateGraph(state_schema).compile(checkpointer = checkpointer)

# 3. 스레드 ID로 상태 관리
config = {"configurable": {"thread_id": "thread-1"}}
result = app.invoke(input_date, config=config)
"""
None

In [4]:
# 기본적으로 제공하는 체크포인터는 다음과 같습니다.

# • BaseCheckpointSaver : 추상 기본 클래스
# • InMemorySaver : 메모리 기반 구현
# • SQLiteSaver, PostgresSaver : 영구 저장소 구현

# import 

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# InMemorySaver : 랭그래프의 체크포인트 시스템 중 가장 기본적인 구현체입니다. 메모리에 상태를 저장하는 방식으로, 
#   다음과 같은 특징을 가집니다.

# • 휘발성 : 프로그램이 종료되면 데이터가 사라집니다.
# • 빠른 속도: 메모리 접근이므로 매우 빠릅니다.
# • 개발/테스트용 : 주로 프로토타이핑과 테스트에 사용됩니다.

# 프로덕션에서는 SQLiteSaver, PostgresSaver 등으로 교체하여 영구 저장할 수 있습니다.

In [7]:
from typing import Dict, Any
from langgraph.graph import StateGraph, START, END

# InMemorySaver 임포트
from langgraph.checkpoint.memory import InMemorySaver

from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
import json


# 상태 정의

In [8]:
class MemoryBotState(BaseModel):
    user_message: str = Field(default="", description="사용자 입력 메세지")
    user_name: str = Field(default="", description="사용자 이름")
    user_preferences: Dict[str, Any] = Field(
        default_factory=dict, description="사용자 선호도"
    )
    response: str = Field(default="", description="최종 응답")

# 메시지 처리 노드

In [9]:
# 사용자의 메세지를 분석아혀 상태를 업데이트 하는 역할.

# ★ InMemorySaver가 노드 실행 전에 자동으로 이전 상태를 로드하며, 노드 실행 후에 자동으로 새 상태를 저장합니다. 
# 그러므로 개발자가 별도로 대화 이력관리를 하지 않아도 됩니다.



In [10]:
llm = ChatOpenAI(model='gpt-4o')

In [11]:
def process_message(state: MemoryBotState) -> Dict[str, Any]:
    message = state.user_message
    user_name = state.user_name
    preferences = state.user_preferences.copy()   # 사본!

    # 시스템 프롬프트
    system_prompt = f"""
당신은 사용자의 정보를 기억하는 메모리 봇입니다.
현재 기억하고 있는 정보:
- 사용자 이름: {user_name if user_name else "모름"}
- 좋아하는 것: {preferences.get("likes", [])}
- 싫어하는 것: {preferences.get("dislikes", [])}

사용자 메시지를 분석하여 다음 JSON 형태로 응답하세요:
{{
  "response": "사용자에게 줄 응답 메시지",
  "new_name": "새로 알게 된 이름 (없으면 null)",
  "new_likes": ["새로 알게 된 좋아하는 것들"],
  "new_dislikes": ["새로 알게 된 싫어하는 것들"]
}}    
    """

    messages = [SystemMessage(content=system_prompt), HumanMessage(content=message)]
    response = llm.invoke(messages)
    print('✅ 응답확인:', response.content)
    result = json.loads(response.content)

    # 새로운 정보 업데이트
    if result.get("new_name"):
        user_name = result['new_name']

    if result.get('new_likes'):
        preferences.setdefault("likes", []).extend(result['new_likes'])

    if result.get("new_dislikes"):
        preferences.setdefault("dislikes", []).extend(result["new_dislikes"])

    bot_response = result.get("response", "죄송해요, 이해하지 못했어요")        


    return {
        "response": bot_response,
        "user_name": user_name,
        "user_preferences": preferences,  # 사본을 세팅함.
    }
    

## .copy() 사본생성

In [12]:
a = {'name': 'John', 'age': 40}
b = a

print(f'a={a}, b={b}')

a={'name': 'John', 'age': 40}, b={'name': 'John', 'age': 40}


In [13]:
a['name'] = 'Susan'

In [14]:
print(f'a={a}, b={b}')

a={'name': 'Susan', 'age': 40}, b={'name': 'Susan', 'age': 40}


In [15]:
b = a.copy()  # dict 의 사본객체 생성
print(f'a={a}, b={b}')

a={'name': 'Susan', 'age': 40}, b={'name': 'Susan', 'age': 40}


In [16]:
a['name'] = "홍길동"
print(f'a={a}, b={b}')

a={'name': '홍길동', 'age': 40}, b={'name': 'Susan', 'age': 40}


# 메모리 봇 그래프 생성

In [17]:
def create_memory_bot_graph():
    checkpointer = InMemorySaver()  # 이를 사용하여 자동 메모리 관리

    workflow = StateGraph(MemoryBotState)

    workflow.add_node("process_message", process_message)

    workflow.add_edge(START, "process_message")
    workflow.add_edge("process_message", END)

    # checkpointer 와 함께 컴파일
    return workflow.compile(checkpointer=checkpointer)

## InMemorySaver 로 자동 메모리 관리

In [18]:
# ⑤ InMemorySaver로 자동 메모리 관리: In MemorySaver의 내부 구조를 매우 단순하게 '표현'하면 다음과 같습니다. 
# 저장소로 사용할 딕셔너리 (storage)를 만들고 거기에 thread_id별로 데이터를 쌓고, 
# 가져올 때도 thread_id를 기준으로 가져옵니다.

"""
class InMemorySaver:
    def __init__(self):
        self.storage = {}  # {thread_id: {checkpoint_id: state}}

    def put(self, config, checkpoint):
        thread_id = config['configurable']['thread_id']
        self.storage[thread_id] = checkpoint

    def get(self, config):
        thread_id = config['configurable']['thread_id']
        return self.storage.get(thread_id)
"""
None


## checkpointer 와 함께 컴파일

In [19]:
"""
checkpointer와 함께 컴파일: compile() 메서드에 체크포인터를 전달하면 다음과 같은 일이 발생합니다.

• 각 노드를 래핑: 체크포인터가 각 노드 실행을 감싸는 래퍼 생성
• 자동 체크포인트: 노드 실행 전후에 자동으로 체크포인트 생성
• 상태 병합: 이전 상태와 새 입력을 자동으로 병합
"""
None

# InMemorySaver 사용을 위한 config 설정

In [20]:
"""
InMemorySaver 사용을 위한 config 설정 : 체크포인트를 사용하려면 thread_id가 필요합니다. 
thread_id 설정은 그래프 실행 시 설정으로 추가하면 됩니다. 
config는 '매 실행마다 전달'되어야 합니다. 

InMemorySarver는 config에 있는 thread_id를 확인하여 어디에 있는 데이터를 불러올지, 
어디에 상태를 저장해야 할지 알 수 있게 됩니다.
"""
None

# 실행

In [21]:
app = create_memory_bot_graph()

thread_id = "orange_123"   # thread_id - 세션 식별자.

# 테스트 대화
conversations = [
    "안녕하세요!",
    "내 이름은 감귤이야",  # 이름 정보
    "김치찜을 좋아해",   # '좋아하는 것' 정보
    "해삼은 싫어해",    # '싫어하는 것' 정보
    # ↓ 메모리의 내용이 답변이 되어야 한다!
    "내 이름이 뭐였지?",
    "내가 좋아하는 것과 싫어하는 것은?",
]

for i, message in enumerate(conversations, 1):
    print(f"[{i}] 사용자: {message}")

    # InMemorySaver 사용을 위한 config 설정
    config = {"configurable": {"thread_id": thread_id}}
    result = app.invoke({"user_message": message}, config)

    print(f"[{i}] 챗봇: {result['response']}")

    print( # 메모리의 내용 확인
        f"메모리: 이름={result.get('user_name', '없슴')}, "
        f"좋아하는 것={result.get("user_preferences", {})}\n"
    )




[1] 사용자: 안녕하세요!
✅ 응답확인: {
  "response": "안녕하세요! 만나서 반가워요. 어떻게 도와드릴까요?",
  "new_name": null,
  "new_likes": [],
  "new_dislikes": []
}
[1] 챗봇: 안녕하세요! 만나서 반가워요. 어떻게 도와드릴까요?
메모리: 이름=, 좋아하는 것={}

[2] 사용자: 내 이름은 감귤이야
✅ 응답확인: {
  "response": "반가워요, 감귤님! 무엇을 도와드릴까요?",
  "new_name": "감귤",
  "new_likes": [],
  "new_dislikes": []
}
[2] 챗봇: 반가워요, 감귤님! 무엇을 도와드릴까요?
메모리: 이름=감귤, 좋아하는 것={}

[3] 사용자: 김치찜을 좋아해
✅ 응답확인: {
  "response": "김치찜을 좋아하시는군요! 기억해둘게요.",
  "new_name": null,
  "new_likes": ["김치찜"],
  "new_dislikes": []
}
[3] 챗봇: 김치찜을 좋아하시는군요! 기억해둘게요.
메모리: 이름=감귤, 좋아하는 것={'likes': ['김치찜']}

[4] 사용자: 해삼은 싫어해
✅ 응답확인: {
  "response": "해삼을 싫어하시는군요! 알겠습니다.",
  "new_name": null,
  "new_likes": [],
  "new_dislikes": ["해삼"]
}
[4] 챗봇: 해삼을 싫어하시는군요! 알겠습니다.
메모리: 이름=감귤, 좋아하는 것={'likes': ['김치찜'], 'dislikes': ['해삼']}

[5] 사용자: 내 이름이 뭐였지?
✅ 응답확인: {
  "response": "당신의 이름은 감귤이에요.",
  "new_name": null,
  "new_likes": [],
  "new_dislikes": []
}
[5] 챗봇: 당신의 이름은 감귤이에요.
메모리: 이름=감귤, 좋아하는 것={'likes': ['김치찜'], 'dislikes': ['해삼'